<a href="https://colab.research.google.com/github/ChiragBellara/Video-Translation-Using-Deep-Learning/blob/silbunsa-patch-1/TTS_with_bark/TTS_bark_mainipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This module would be used for converting Text to Speech. I have currently used suno-bark(https://github.com/suno-ai/bark) as it uses transformer for the voice and supports 13 languages. As a video can have lengthy speech I have used nltk for splitting them into sentences. Initially I am taking script.txt as the speech that needs to be converted and saving it as a tts.wav file.

This module needs to work as an API getting the speech and outputting the audio file for desired language.

In [ ]:
!pip install git+https://github.com/suno-ai/bark.git

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-odcc3g2f
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-odcc3g2f
  Resolved https://github.com/suno-ai/bark.git to commit 773624d26db84278a55aacae9a16d7b25fbccab8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ww08ck5l
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-ww08ck5l
  Resolved https://github.com/huggingface/transformers.git to commit 6cbc1369a330860c128a1ba365f246751382c9e5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio
import scipy
# download and load all models
preload_models()


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import nltk  # we'll use this to split into sentences
import numpy as np
import soundfile as sf

nltk.download('punkt')
from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
script_file = "script.txt"
with open(script_file, "r") as file:
  script = file.read()
script.replace("\n", " ").strip()

'नई दिल्ली : मिलेट्स वर्ष में योगी सरकार श्रीअन्न को बढ़ावा दे रही है. इसी क्रम में 27 से 29 अक्टूबर तक इंदिरा गांधी प्रतिष्ठान में श्रीअन्न महोत्सव का आयोजन किया जा रहा है. यहां एक तरफ जहां श्रीअन्न के पोषक मूल्य खेती- प्रसंस्करण पर चर्चा होगी, वहीं दूसरी तरफ लखनऊ के नामचीन होटल व रेस्तरां भी मिलेट्स व्यंजनों की कुकिंग प्रतियोगिता में हिस्सा लेंगे. इसके लिए आयोजन के दूसरे दिन चटोरी गली में यह प्रतियोगिता प्रस्तावित है. मोटे अनाजों को बढ़ावा देने के योगी सरकार के निर्देश के उपरांत यह तैयारी की जा रही है. चटोरी गली में मिलेट्स उत्पाद की खुशबू बिखेरेंगे होटल-रेस्तरां कृषि विभाग की देखरेख में होने वाले श्रीअन्न महोत्सव के अंतर्गत मिलेट्स व्यंजनों की प्रतियोगिता होगी. इसमें लखनऊ के कई नामचीन होटल व रेस्तरां की तरफ से मिलेट्स व्यंजन आमजन के लिए होंगे. कृषि विभाग व एलडीए के संयुक्त तत्वावधान में यह कार्यक्रम प्रस्तावित है. चटोरी गली में होने वाली प्रतियोगिता के लिए होटल-रेस्तरां एसोसिएशन से संपर्क साधा जा रहा है. इसमें बाजरा, कुट्टू, रामदाना, ज्वार, कोदो, सावां, ज्वार आदि के अनेक लजीज व्यंजन

In [ ]:
def convert(speech,lang):
  sentences = nltk.sent_tokenize(script)
  speaker = {
    "en": "v2/en_speaker_3",
    "hi": "v2/hi_speaker_2",
    "ja": "v2/ja_speaker_6",
    "fr": "v2/fr_speaker_6",
    "es": "v2/es_speaker_6"
    }

  SPEAKER = speaker[lang]
  GEN_TEMP = 0.6
  silence = np.zeros(int(0.2 * SAMPLE_RATE))  # quarter second of silence
  pieces = []
  for sentence in sentences:
    semantic_tokens = generate_text_semantic(
        sentence, #Tokenized sentence
        history_prompt=SPEAKER, #speaker model
        temp=GEN_TEMP, #Threshold for randomness in generated voice
        min_eos_p=0.05,  # this controls how likely the generation is to end
    )
    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)
    pieces += [audio_array, silence.copy()]

  final_piece=np.concatenate(pieces)
  sf.write("output_audio.wav", final_piece, SAMPLE_RATE)
  # Get the full path to the saved file
  file_path = os.path.abspath("output_audio.wav")
  return file_path



In [ ]:
converted_path=convert(script,"hi")


100%|██████████| 32/32 [00:30<00:00,  1.05it/s]
